## Running and debugging

Debugging in notebooks sucks. It's <strike>best</strike> better to use pdb inside emacs:

1. Copy the code to a python file
2. Run using pdb remotely via tramp+anaconda
3. **Remember to set breakpoints in the files inside** `~/local/lib/python2.7/site-packages/` !!!

## Current

* Remember to work both in `quadraturetransformer.py` and `optimisedquadraturetransformer.py`.
* Maybe you have to pass the whole `fiat_element.mapping()` array around?

## Some definitions

**Multiindex: ** An array like `[[0, 0], [0, 1], [1, 0], [1, 1]]`, where each element is a list of derivatives and each element in a derivative is the index of the spatial dimension along which it is taken. In the example we have $\partial^2_x , \partial_x \partial_y , \partial_y \partial_x $ and $ \partial^2_y$

# Misc tests

In [ ]:
from dolfin import *
# create mesh and define function space
mesh = UnitCubeMesh(32, 32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)
# define Dirichlet Boundary at bottom face
def boundary(x):
    return x[0] < DOLFIN_EPS

# define boundary condition
bc = DirichletBC(V, Constant(0.0), boundary)
# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression("10 * exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2) + pow(x[2] - 0.5, 2)) / 0.02)", degree=1)
a = inner(grad(u), grad(v)) * dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [ ]:
plot(u, cmap='bone')

In [ ]:
print("This sometimes screws the notebook...")
#from IPython.core.display import HTML

#HTML(X3DOM.html(u))

# DOFs & co

Don't forget these...

In [ ]:
V = FunctionSpace(UnitIntervalMesh(4), "Lagrange", 1)
u = interpolate(Expression("x[0]**2", degree=1), V)

# These only work on spaces where every vertex has exactly one dof
dof_to_vertex_map(V)
vertex_to_dof_map(V)

# How to get expansion coefs on cell.
# An alternative to this is u.restrict()
cell_id = 1
dofs = V.dofmap().cell_dofs(cell_id)
dofs = u.vector()[dofs]

# Building sparse matrices with PETSc4Py

See:
* https://github.com/pyHPC/pyhpc-tutorial/blob/master/markdown/scale/petsc4py-tutorial.md
* https://www.bu.edu/pasi/files/2011/01/Lisandro-Dalcin-petsc4py.pdf
* [This demo](https://bitbucket.org/cwilson/dolfin/src/883fcc4a12d53c7dae1bb9d5fc7e7302051b0bae/demo/undocumented/petsc4py/python/demo_petsc4py.py?at=master&fileviewer=file-view-default)

There is a [bug](https://bugs.launchpad.net/dolfin/+bug/1063868) in the automatically generated documentation which affects [`GenericMatrix.set()`](https://fenicsproject.org/olddocs/dolfin/2016.2.0/python/programmers-reference/cpp/la/GenericMatrix.html?highlight=genericmatrix#dolfin.cpp.la.GenericMatrix.set) and `get()`. Only one function with three parameters is actually exposed by SWIG and the second and last are actually the indices of the rows and columns to modify.

In [ ]:
from petsc4py import PETSc
import numpy as np

A = PETSc.Mat().create()
A.setSizes([100, 100])
#A.setType('python')
#shell = Del2Mat(n) # what is this?
#A.setPythonContext(shell)
A.setType("aij")
A.setUp()

A.setValues([1,2,3], [0,5,9], np.eye(3))
A.assemble()

B = A.convert("dense")
B.getDenseArray()

## Inserting new data into PETSc matrices

If we try to to `set()` or `add()` rows/cols of an already initialised sparse PETSc matrix, PETSc can complain with an "out of range" error (code #63) if the insertions don't agree with the sparsity pattern. From the [mailing list](http://lists.mcs.anl.gov/pipermail/petsc-users/2012-February/012242.html):

    "Preallocation routines now automatically set MAT_NEW_NONZERO_ALLOCATION_ERR,
    if you intentionally preallocate less than necessary then use
    MatSetOption(mat,MAT_NEW_NONZERO_ALLOCATION_ERR,PETSC_FALSE) to disable the
    error generation"

Here are some constants: [PETScBool](http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/Sys/PetscBool.html), [MatOption](http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/Mat/MatOption.html), [PETSc error codes](https://www.mcs.anl.gov/petsc/petsc-current/include/petscerror.h.html).

In [ ]:
# Inject necessary constants into petsc4py's "namespace"
PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR = 19
PETSc.PETSc_FALSE = 0

# mock example:
A = assemble(a)
#A.set(block, row_indices, col_indices)   # error
A.mat().setOption(PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR, PETSc.FALSE)
A.set(block, row_indices, col_indices)   # all good
A.apply('insert')

# Periodic boundary conditions

In [ ]:
from dolfin import *
from ffc.log import add_logfile, set_level, DEBUG
set_level(DEBUG)
add_logfile("/tmp/fenics.log")

class PeriodicBoundary(SubDomain):

    def inside(self, x, on_boundary):
        return bool((near(x[0], 0) or near(x[1], 0)) and 
                (not ((near(x[0], 0) and near(x[1], 1)) or 
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:  
            y[0] = x[0]
            y[1] = x[1] - 1.

k = 5
n = 2**k 
mesh = UnitSquareMesh(n, n)
V = VectorFunctionSpace(mesh, 'CG', 1, constrained_domain = PeriodicBoundary())

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
s = Constant("1.0")
uex = Expression(("-cos(2*DOLFIN_PI*x[1])","-sin(2*DOLFIN_PI*x[0])"), element=V.ufl_element())
t = Expression(("4*DOLFIN_PI*DOLFIN_PI*cos(2*DOLFIN_PI*x[1])","4*DOLFIN_PI*DOLFIN_PI*sin(2*DOLFIN_PI*x[0])"), element = V.ufl_element())
a = -s*inner(grad(u), grad(v))*dx
L = +s*div(v)*dx + inner(t, v)*dx

u = Function(V)

In [ ]:
problem = LinearVariationalProblem(a, L, u, bcs=[])

In [ ]:
solver = LinearVariationalSolver(problem) 
solver.parameters["linear_solver"] = "gmres"
solver.parameters["preconditioner"] = "hypre_amg"
solver.parameters["krylov_solver"]["monitor_convergence"] = False
solver.parameters["krylov_solver"]["relative_tolerance"] = 1e-10

In [ ]:
solver.solve()

In [ ]:
file = File("periodic_trial_%i.pvd" % k)
file << u
file2 = File("periodic_exact.pvd")
u_exact =interpolate(uex, V) 
file2 << u_exact
print("error %f" % assemble(inner(u-u_exact, u-u_exact) * dx))